In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import cv2
import pathlib

In [2]:
DIR_TRAIN = "C:/Users/SURFACE PRO/Desktop/3rd challenge/farmy-challenge-haick-2022/train/"
DIR_TEST = "C:/Users/SURFACE PRO/Desktop/3rd challenge/farmy-challenge-haick-2022/test/"
DIR_VAL = "C:/Users/SURFACE PRO/Desktop/3rd challenge/farmy-challenge-haick-2022/valid/"

DIR_TRAIN = pathlib.Path(DIR_TRAIN)
DIR_VAL = pathlib.Path(DIR_VAL)

In [3]:
test_imgs = []
noms = []
for img in os.listdir(DIR_TEST):
    noms.append(img)
    test_imgs.append(load_img(DIR_TEST + img))
#test_img = np.asarray(test_imgs)

batch_size = 32
img_height = 300
img_width = 300
dim = (img_width,img_height)

In [4]:
"""""for img in test_imgs:
    img = cv2.resize(img,dim)"""

train_ds = tf.keras.utils.image_dataset_from_directory(
  DIR_TRAIN,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
  DIR_VAL,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 2474 files belonging to 3 classes.
Using 1980 files for training.
Found 625 files belonging to 3 classes.
Using 125 files for validation.


In [7]:
class_names = train_ds.class_names

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

(32, 300, 300, 3)
(32,)
0.0 0.9527155


In [11]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [12]:
model = Sequential([
  data_augmentation,  
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(3)
])

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
62/62 [==============================] - 111s 2s/step - loss: 1.3068 - accuracy: 0.5672 - val_loss: 0.8531 - val_accuracy: 0.6480
Epoch 2/15
62/62 [==============================] - 109s 2s/step - loss: 0.6835 - accuracy: 0.7283 - val_loss: 0.7021 - val_accuracy: 0.6800
Epoch 3/15
62/62 [==============================] - 110s 2s/step - loss: 0.5627 - accuracy: 0.7616 - val_loss: 0.5767 - val_accuracy: 0.6960
Epoch 4/15
62/62 [==============================] - 113s 2s/step - loss: 0.4730 - accuracy: 0.8091 - val_loss: 0.4374 - val_accuracy: 0.7920
Epoch 5/15
62/62 [==============================] - 115s 2s/step - loss: 0.4189 - accuracy: 0.8379 - val_loss: 0.3923 - val_accuracy: 0.8480
Epoch 6/15
62/62 [==============================] - 201s 3s/step - loss: 0.3992 - accuracy: 0.8449 - val_loss: 0.3911 - val_accuracy: 0.8640
Epoch 7/15
62/62 [==============================] - 471s 8s/step - loss: 0.3890 - accuracy: 0.8399 - val_loss: 0.4931 - val_accuracy: 0.7760
Epoch 8/15
62

In [15]:
res = []
i = 0
for i in range(len(test_imgs)) :
    img_array = img_to_array(test_imgs[i])
    img_array = tf.image.resize(img_array, [300,300])
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print(class_names[np.argmax(score)])
    res.append(class_names[np.argmax(score)])

early blight
late blight
healthy
healthy
late blight
healthy
late blight
late blight
healthy
healthy
late blight
healthy
healthy
healthy
healthy
late blight
healthy
late blight
healthy
late blight
late blight
late blight
late blight
late blight
late blight
healthy
healthy
late blight
healthy
healthy
healthy
healthy
late blight
healthy
late blight
healthy
healthy
healthy
healthy
healthy
healthy
healthy
late blight
late blight
late blight
healthy
healthy
late blight
healthy
late blight
healthy
healthy
late blight
late blight
healthy
late blight
healthy
healthy
late blight
late blight
late blight
healthy
healthy
healthy
late blight
late blight
healthy
late blight
late blight
early blight
healthy
late blight
healthy
late blight
late blight
healthy
late blight
late blight
late blight
early blight
healthy
late blight
late blight
healthy
late blight
healthy
late blight
healthy
healthy
healthy
late blight
late blight
healthy
late blight
healthy
healthy
healthy
healthy
late blight
healthy
late 

In [16]:
df = pd.DataFrame(list(zip(noms,res)), columns=['image','Classification target'])
df.to_csv('subs.csv',index=False)